In [1]:
import tensorflow as tf

c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\shubh\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\shubh\appd

In [2]:
print(tf.__version__)

1.14.0


In [23]:
x = tf.Variable(3 , name="x")
y = tf.Variable(4 , name="y")

In [24]:
f = x*x*y + y+2

In [25]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)


42


In [26]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result1 = f.eval()
    

In [27]:
result1

42

In [28]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result2 = f.eval()

In [29]:
result2

42

In [30]:
sess = tf.InteractiveSession()
init.run()
result3= f.eval()
print(result3)
sess.close()

42


In [31]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

##any node or var is generally added to the default graph

True

In [32]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    

In [33]:
x2.graph is graph

True

In [34]:
x2.graph is tf.get_default_graph()

False

In [35]:
w = tf.constant(3)
x = w+2
y = x+5
z = y*x


In [36]:
with tf.Session() as sess:
    print(z.eval())
    print(y.eval())

50
10


In [37]:
with tf.Session() as sess:
    y,z = sess.run([y,z])
    print(y)
    print(z)

10
50


# linear regression with tf

In [38]:
import numpy as np
from sklearn.datasets import fetch_california_housing


housing_data = fetch_california_housing()

In [39]:
import joblib

joblib.dump(housing_data , "housing_data.pkl")


['housing_data.pkl']

In [40]:
 m, n = housing_data.data.shape
 m,n    
 
    

(20640, 8)

In [41]:
housing_data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n--------------------

In [42]:
housing_data_with_bias = np.c_[np.ones((m,1)) , housing_data.data]

In [ ]:
x=tf.constant(housing_data_with_bias , dtype = tf.float32 , name = "x")
y= tf.constant(housing_data.target.reshape(-1,1) , dtype = tf.float32 , name = "y")
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt , x)) , xt) , y)

In [ ]:
with tf.Session() as sess:
    thetha_val = theta.eval()
    print(thetha_val.shape)

In [8]:
n_epochs = 1000
learning_rate = 0.01


In [9]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()

scaled_housing_x = scalar.fit_transform(housing_data.data)
scaled_housing_x = np.c_[np.ones((m,1)) , scaled_housing_x]

In [43]:
x_scaled=tf.constant(scaled_housing_x , dtype = tf.float32 , name = "x")
y_scaled= tf.constant(housing_data.target.reshape(-1,1) , dtype = tf.float32 , name = "y")

theta_new = tf.Variable(tf.random_uniform([n+1,1] , -1.0 ,1.0 , seed=42) , name = "theta" )
y_pred_new = tf.matmul(x_scaled , theta_new , name ="predict")
error_new  = y_pred_new - y_scaled

In [44]:
mse_new =  tf.reduce_mean(tf.square(error_new) , name= "mse")
gradient_new = 2/m * tf.matmul(tf.transpose(x_scaled) , error_new)

training_op = tf.assign(theta_new , theta_new - learning_rate * gradient_new)



In [45]:
init = tf.global_variables_initializer()

In [46]:
with tf.Session() as sess:
    init.run()
    print(y_scaled)
    for epocs in range(n_epochs):
        training_op.eval()
        sess.run(training_op)

        
        if epocs % 100 == 0:
            print("epoch" , epocs, "mse= " , mse_new.eval())
#             print(theta_new.eval() )
#             print("epoch" , epocs, "error= " , error_new.eval())


    best_theta = theta_new.eval()        

Tensor("y_4:0", shape=(20640, 1), dtype=float32)
epoch 0 mse=  2.5812879


In [47]:
best_theta

array([[ 1.2914524 ],
       [ 0.45051962],
       [ 0.48726475],
       [ 0.4376735 ],
       [-0.13772976],
       [ 0.225732  ],
       [-0.43742767],
       [-0.686779  ],
       [-0.4399947 ]], dtype=float32)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing_data.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [48]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_data.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients_test = tf.gradients(mse , theta)
gradients = tf.gradients(mse , theta)[0]



# training_op = tf.assign(theta, theta - learning_rate * gradients)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
    
        if epoch % 100 == 0:
            print(gradients_test[0].eval())
#             print("Epoch", epoch, "MSE =", mse.eval())
#             print(gradients.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)

[[-2.3280308 ]
 [-0.56650347]
 [ 0.58552176]
 [ 0.46400136]
 [ 0.5997245 ]
 [ 0.18164141]
 [-1.1452663 ]
 [-0.6055829 ]
 [ 0.7586533 ]]
[[ 1.1174731 ]
 [ 0.40708742]
 [ 0.53542376]
 [ 0.47013983]
 [-0.10018849]
 [ 0.24383584]
 [-0.52316684]
 [-0.7205194 ]
 [-0.39300156]]


In [53]:
def my_func(a,b):
    z=0
    for i in range(100):
        z = a*np.cos(z+i) + z*np.sin(b-i)
    return z    

In [54]:
my_func(30000,4000)

51241.335635307245

In [49]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

# Mini Batch gradient descent

In [50]:
tf.reset_default_graph()

In [51]:

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[6. 7. 8.]]


In [52]:
 B_val_2

array([[ 9., 10., 11.],
       [12., 13., 14.]], dtype=float32)

In [53]:
x_mini_batch = tf.placeholder(tf.float32 , shape = (None , n+1) , name = "x")
y_mini_batch = tf.placeholder(tf.float32 , shape = (None , 1) , name = "y")

In [54]:
batch_size = 100
n_batches = int(np.ceil(m/batch_size))
n_epochs = 100

In [55]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(x_mini_batch, theta, name="predictions")
error = y_pred - y_mini_batch
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients_test = tf.gradients(mse , theta)
gradients = tf.gradients(mse , theta)[0]



# training_op = tf.assign(theta, theta - learning_rate * gradients)
training_op = optimizer.minimize(mse)

saver_node = tf.train.Saver()

init = tf.global_variables_initializer()


In [56]:
def feed_batch(epoch , batch_index , batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing_data.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

In [57]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch%10 == 0:
            save_path = saver_node.save(sess, "/tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = feed_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={x_mini_batch: X_batch, y_mini_batch: y_batch})

    best_theta = theta.eval()
    save_path = saver_node.save(sess, "/tmp/my_model_final.ckpt")


In [58]:
with tf.Session() as sess:
    saver_node.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [59]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_log_dir = "tf_logs"
log_dir = "{}/run-{}/".format(root_log_dir,now)

In [60]:
mse_summary = tf.summary.scalar("mse" , mse)
file_writer = tf.summary.FileWriter(log_dir , tf.get_default_graph())

In [61]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [62]:
with tf.Session() as sess:
    init.run()
    for epoch in range (n_epochs):
        for batch_index in range(n_batches):
            x_batch , y_batch = feed_batch(epoch , batch_index , batch_size)
            if batch_index%10 == 0:
                summary_str = mse_summary.eval(feed_dict= {x_mini_batch : x_batch , y_mini_batch : y_batch})
                step = epoch * n_batches + batch_index
                
                file_writer.add_summary(summary_str , step)
        sess.run(training_op, feed_dict={x_mini_batch: x_batch, y_mini_batch: y_batch})
        
    best_theta = theta.eval()              
                
                

In [63]:
file_writer.close()

In [20]:
def relu(x):
    w_shape = (int(x.get_shape()[1]) ,1)
    w = tf.Variable(tf.random_normal(w_shape) , name = "weights")
    b = tf.Variable(0.0 , name = "bias")
    z = tf.add(tf.matmul(x,w) , b , name = "z")
    return tf.maximum(z,0.0 , name = "relu")


In [65]:
n_features = 3
x = tf.placeholder(tf.float32 , shape = (None , n_features ) , name = "x")


w1 = tf.Variable(tf.random_normal((n_features , 1) ) , name = "weights1")
w2 = tf.Variable(tf.random_normal((n_features , 1) ) , name = "weights2")

b1 = tf.Variable(0.0 , name = "bias1")
b2 = tf.Variable(0.0 , name = "bias2")

z1 = tf.add(tf.matmul(x , w1) , b1 , name ="z1")
z2 = tf.add(tf.matmul(x , w2) , b2 , name ="z2")

relu1 = tf.maximum(z1 ,0 ,name = "relu1")
relu2 = tf.maximum(z2 ,0 ,name = "relu2")

output = tf.add(relu1 , relu2 , name = "output")

In [69]:
relus = [ relu(x) for i in range(5)]
relus
# output = tf.add_n(relus , name = "output")

[None, None, None, None, None]